In [1]:
import pandas
import inspect

In [2]:
source_DF = inspect.getsource(pandas.DataFrame)
print(type(source_DF))

<class 'str'>


In [3]:
print(source_DF[:200])

class DataFrame(NDFrame):
    """
    Two-dimensional, size-mutable, potentially heterogeneous tabular data.

    Data structure also contains labeled axes (rows and columns).
    Arithmetic operation


In [4]:
from sagas.tests.brokers import faust_task
# faust_task.add
src=inspect.getsource(faust_task.AddOperation)
print(src)

class AddOperation(faust.Record):
    x: int
    y: int



In [5]:
from types import FunctionType

def extract_wrapped(decorated):
    closure = (c.cell_contents for c in decorated.__closure__)
    return next((c for c in closure if isinstance(c, FunctionType)), None)

In [6]:
inspect.unwrap(faust_task.add)

<Agent: init sagas.tests.brokers.faust_[.]add>

In [7]:
sig = inspect.signature(faust_task.add, follow_wrapped=True)
sig

<Signature (*, index: int = None, active_partitions: Set[faust.types.tuples.TP] = None, stream: faust.types.streams.StreamT = None, channel: faust.types.channels.ChannelT = None) -> faust.types.agents.ActorT[typing.Union[typing.AsyncIterable, typing.Awaitable]]>

In [8]:
import dill
src=dill.source.getsourcelines(faust_task.add)
src[0][:10]

['class Agent(AgentT, Service):\n',
 '    """Agent.\n',
 '\n',
 '    This is the type of object returned by the ``@app.agent`` decorator.\n',
 '    """\n',
 '\n',
 '    # supervisor is None until the agent is started so we cast to simplify.\n',
 '    supervisor: SupervisorStrategyT = cast(SupervisorStrategyT, None)\n',
 '\n',
 '    # channel is loaded lazily on .channel property access\n']

In [9]:
import ast, gast
code = open('/Users/xiaofeiwu/jcloud/assets/langs/workspace/rasa/stack/sagas/tests/brokers/faust_task.py').read()
code

'import faust\n\napp = faust.App(\'myapp\', broker=\'kafka://\')\n\nclass AddOperation(faust.Record):\n    x: int\n    y: int\n\n@app.agent()\nasync def add(stream):\n    """\n    Add function\n    :param stream:\n    :return:\n    """\n    async for op in stream:\n        yield op.x + op.y\n\n@app.command()\nasync def produce():\n    await add.send(value=AddOperation(2, 2))\n\nif __name__ == \'__main__\':\n    # $ faust -A sagas.tests.brokers.faust_task worker -l info\n    # $ faust -A sagas.tests.brokers.faust_task produce\n    app.main()\n'

In [10]:
tree = ast.parse(code)
gtree = gast.ast_to_gast(tree)
dump = gast.dump(gtree)
dump

"Module(body=[Import(names=[alias(name='faust', asname=None)]), Assign(targets=[Name(id='app', ctx=Store(), annotation=None)], value=Call(func=Attribute(value=Name(id='faust', ctx=Load(), annotation=None), attr='App', ctx=Load()), args=[Str(s='myapp')], keywords=[keyword(arg='broker', value=Str(s='kafka://'))])), ClassDef(name='AddOperation', bases=[Attribute(value=Name(id='faust', ctx=Load(), annotation=None), attr='Record', ctx=Load())], keywords=[], body=[AnnAssign(target=Name(id='x', ctx=Store(), annotation=None), annotation=Name(id='int', ctx=Load(), annotation=None), value=None, simple=1), AnnAssign(target=Name(id='y', ctx=Store(), annotation=None), annotation=Name(id='int', ctx=Load(), annotation=None), value=None, simple=1)], decorator_list=[]), AsyncFunctionDef(name='add', args=arguments(args=[Name(id='stream', ctx=Param(), annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Expr(value=Str(s='\\n    Add function\\n    :param stream:\\

In [11]:
{name for name, _ in gast.iter_fields(tree)}

{'body'}

In [12]:
tree.body[0], tree.body[1]

(<_ast.Import at 0x7feb030ee750>, <_ast.Assign at 0x7feb030ee7d0>)

In [13]:
tree.body[3]

In [14]:
func=gtree.body[3]
# docs = gast.get_docstring(func)
# docs
gast.dump(func)

"AsyncFunctionDef(name='add', args=arguments(args=[Name(id='stream', ctx=Param(), annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Expr(value=Str(s='\\n    Add function\\n    :param stream:\\n    :return:\\n    ')), AsyncFor(target=Name(id='op', ctx=Store(), annotation=None), iter=Name(id='stream', ctx=Load(), annotation=None), body=[Expr(value=Yield(value=BinOp(left=Attribute(value=Name(id='op', ctx=Load(), annotation=None), attr='x', ctx=Load()), op=Add(), right=Attribute(value=Name(id='op', ctx=Load(), annotation=None), attr='y', ctx=Load()))))], orelse=[])], decorator_list=[Call(func=Attribute(value=Name(id='app', ctx=Load(), annotation=None), attr='agent', ctx=Load()), args=[], keywords=[])], returns=None)"

In [15]:
import astunparse
class Trans(gast.NodeTransformer):
    def visit_AsyncFunctionDef(self, node):
        print(node.name)
        print(astunparse.unparse(node))
        return node.name

result = Trans().visit(gtree)
# gast.dump(result)

add


@app.agent()
async def add(stream):
    '\n    Add function\n    :param stream:\n    :return:\n    '
    async for op in stream:
        (yield (op.x + op.y))

produce


@app.command()
async def produce():
    (await add.send(value=AddOperation(2, 2)))



In [16]:
tree.body[2]

In [17]:
# compile(gast.gast_to_ast(tree), '<test>', 'exec')

In [18]:
code = 'with open("any"): pass'
tree_c = gast.parse(code)
compile(gast.gast_to_ast(tree_c), '<test>', 'exec')

<code object <module> at 0x7feb03143c90, file "<test>", line 1>

In [19]:
import astunparse
astunparse.unparse(gast.gast_to_ast(tree_c))

"\nwith open('any'):\n    pass\n"

In [20]:
print(astunparse.unparse(tree.body[2]))



class AddOperation(faust.Record):
    x: int
    y: int



In [21]:
tree.body[3], tree.body[3].body[0]

(<_ast.AsyncFunctionDef at 0x7feb030eec10>, <_ast.Expr at 0x7feb030eecd0>)

In [22]:
gast.dump(tree.body[1]), type(tree.body[1].targets[0])

("Assign(targets=[Name(id='app', ctx=Store())], value=Call(func=Attribute(value=Name(id='faust', ctx=Load()), attr='App', ctx=Load()), args=[Str(s='myapp')], keywords=[keyword(arg='broker', value=Str(s='kafka://'))]))",
 _ast.Name)

In [23]:
type(tree.body[3])

_ast.AsyncFunctionDef

In [24]:
node=tree.body[3]
{name for name, _ in gast.iter_fields(node)}, node.name

({'args', 'body', 'decorator_list', 'name', 'returns'}, 'add')

In [25]:
tree._fields

('body',)

In [26]:
for node in tree.body:
    print({name for name, _ in gast.iter_fields(node)})
    if 'name' in node._fields:
        print(node.name)
    else:
        print(type(node).__name__)
    

{'names'}
Import
{'value', 'targets'}
Assign
{'keywords', 'decorator_list', 'bases', 'body', 'name'}
AddOperation
{'args', 'decorator_list', 'returns', 'body', 'name'}
add
{'args', 'decorator_list', 'returns', 'body', 'name'}
produce
{'orelse', 'body', 'test'}
If


In [27]:
tree

In [28]:
from anytree import NodeMixin
from anytree import RenderTree
class AstNode(NodeMixin, object):
    def __init__(self, parent=None, ast_node=None):
        self.name=ast_node.name if 'name' in ast_node._fields else '_'
        self.parent = parent
        self.ast_node=ast_node       
        self.ast_type=type(ast_node).__name__
        def as_children(body):
            return [AstNode(parent=self, ast_node=n) for n in ast_node.body]
        self.children = as_children(ast_node) if 'body' in ast_node._fields else []

    def __repr__(self):
        return f"{self.name}"

root=AstNode(ast_node=tree)
rnd_tree = RenderTree(root)
for pre, fill, node in rnd_tree:
    print(f"{pre}{node.ast_type}: {node.name}")

Module: _
├── Import: _
├── Assign: _
├── ClassDef: AddOperation
│   ├── AnnAssign: _
│   └── AnnAssign: _
├── AsyncFunctionDef: add
│   ├── Expr: _
│   └── AsyncFor: _
│       └── Expr: _
├── AsyncFunctionDef: produce
│   └── Expr: _
└── If: _
    └── Expr: _
